<a href="https://colab.research.google.com/github/hamza74372/Task2_Text-Sentiment-Analysis/blob/main/Text_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_datasets --quiet

import tensorflow_datasets as tfds
import pandas as pd

# Load IMDB reviews dataset
dataset, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

train_data, test_data = dataset['train'], dataset['test']

# Convert to Pandas DataFrame for easier handling
def tfds_to_df(tf_dataset):
    texts = []
    labels = []
    for text, label in tf_dataset:
        texts.append(text.numpy().decode('utf-8'))
        labels.append(label.numpy())
    return pd.DataFrame({'review': texts, 'sentiment': labels})

train_df = tfds_to_df(train_data)
test_df = tfds_to_df(test_data)

print("Train data shape:", train_df.shape)
print("Test data shape:", test_df.shape)

train_df.head()


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.E9HF39_1.0.0/imdb_reviews-train.tfrecor…

...

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.E9HF39_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.E9HF39_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Train data shape: (25000, 2)
Test data shape: (25000, 2)


,review,sentiment
0,This was an absolutely terrible movie. Don't b...,0
1,"I have been known to fall asleep during films,...",0
2,Mann photographs the Alberta Rocky Mountains i...,0
3,This is the kind of film for a snowy Sunday af...,1
4,"As others have mentioned, all the women that g...",1


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # remove special chars/numbers
    words = text.split()
    words = [w for w in words if w not in stop_words]
    words = [lemmatizer.lemmatize(w) for w in words]
    return ' '.join(words)

# Apply preprocessing to train and test sets
train_df['cleaned_review'] = train_df['review'].apply(preprocess_text)
test_df['cleaned_review'] = test_df['review'].apply(preprocess_text)

train_df.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


,review,sentiment,cleaned_review
0,This was an absolutely terrible movie. Don't b...,0,absolutely terrible movie dont lured christoph...
1,"I have been known to fall asleep during films,...",0,known fall asleep film usually due combination...
2,Mann photographs the Alberta Rocky Mountains i...,0,mann photograph alberta rocky mountain superb ...
3,This is the kind of film for a snowy Sunday af...,1,kind film snowy sunday afternoon rest world go...
4,"As others have mentioned, all the women that g...",1,others mentioned woman go nude film mostly abs...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Vectorize text using TF-IDF
tfidf = TfidfVectorizer(max_features=5000)  # limit features for speed
X_train = tfidf.fit_transform(train_df['cleaned_review'])
X_test = tfidf.transform(test_df['cleaned_review'])

y_train = train_df['sentiment']
y_test = test_df['sentiment']

# Train logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.87704

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.88      0.88     12500
           1       0.88      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



In [ ]:
def predict_sentiment(text):
    text = preprocess_text(text)
    vect = tfidf.transform([text])
    pred = model.predict(vect)
    return "Positive" if pred[0] == 1 else "Negative"

print(predict_sentiment("I really enjoyed this movie!"))
print(predict_sentiment("The film was terrible and boring."))


Positive
Negative
